# 파이프라인 로드

확산 시스템은 매개변수화된 모델 및 스케줄러와 같은 여러 구성 요소로 구성되며 복잡한 방식으로 상호 작용합니다. 이것이 바로 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)이 전체 확산 시스템의 복잡성을 사용하기 쉬운 API로 래핑하도록 설계된 이유입니다. 동시에 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)은 완전히 사용자 지정 가능하므로 각 구성 요소를 수정하여 사용 사례에 맞는 확산 시스템을 구축할 수 있습니다.

이 가이드에서는 다음을 로드하는 방법을 보여줍니다.

- Hub 및 로컬의 파이프라인
- 파이프라인에 다른 구성 요소
- 메모리 사용량을 늘리지 않고 여러 파이프라인
- 다른 부동 소수점 유형 또는 비지수 평균(EMA) 가중치와 같은 체크포인트 변형

## 파이프라인 로드

> [!TIP]
> [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스의 작동 방식에 대한 설명에 관심이 있다면 [DiffusionPipeline 설명](#diffusionpipeline-explained) 섹션으로 건너뛰십시오.

작업에 대한 파이프라인을 로드하는 방법에는 두 가지가 있습니다.

1. 일반 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스를 로드하고 체크포인트에서 올바른 파이프라인 클래스를 자동으로 감지하도록 합니다.
2. 특정 작업에 대한 특정 파이프라인 클래스를 로드합니다.

<hfoptions id="pipelines">
<hfoption id="generic pipeline">

[DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스는 [Hub](https://huggingface.co/models?library=diffusers&sort=trending)에서 최신 트렌드 확산 모델을 로드하는 간단하고 일반적인 방법입니다. [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드를 사용하여 체크포인트에서 작업에 대한 올바른 파이프라인 클래스를 자동으로 감지하고 필요한 모든 구성 및 가중치 파일을 다운로드하고 캐시하며 추론 준비가 된 파이프라인을 반환합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", use_safetensors=True)

이 동일한 체크포인트는 이미지 대 이미지 작업에도 사용할 수 있습니다. [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline) 클래스는 적절한 입력을 제공하는 한 모든 작업을 처리할 수 있습니다. 예를 들어 이미지 대 이미지 작업의 경우 파이프라인에 초기 이미지를 전달해야 합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", use_safetensors=True)

init_image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/img2img-init.png")
prompt = "Astronaut in a jungle, cold color palette, muted colors, detailed, 8k"
image = pipeline("Astronaut in a jungle, cold color palette, muted colors, detailed, 8k", image=init_image).images[0]

</hfoption>
<hfoption id="specific pipeline">

이미 알고 있는 경우 특정 파이프라인 클래스로 체크포인트를 로드할 수 있습니다. 예를 들어 Stable Diffusion 모델을 로드하려면 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline) 클래스를 사용합니다.

In [ ]:
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", use_safetensors=True)

이 동일한 체크포인트는 이미지 대 이미지와 같은 다른 작업에도 사용할 수 있습니다. 체크포인트를 사용할 작업을 구별하려면 해당 작업별 파이프라인 클래스를 사용해야 합니다. 예를 들어 이미지 대 이미지에 동일한 체크포인트를 사용하려면 [StableDiffusionImg2ImgPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/img2img#diffusers.StableDiffusionImg2ImgPipeline) 클래스를 사용합니다.

In [ ]:
from diffusers import StableDiffusionImg2ImgPipeline

pipeline = StableDiffusionImg2ImgPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", use_safetensors=True)

</hfoption>
</hfoptions>

아래 공간을 사용하여 파이프라인을 다운로드하고 로드하기 전에 파이프라인의 메모리 요구 사항을 측정하여 하드웨어에서 실행되는지 확인하십시오.

<div class="block dark:hidden">
	<iframe
        src="https://diffusers-compute-pipeline-size.hf.space?__theme=light"
        width="850"
        height="1600"
    ></iframe>
</div>
<div class="hidden dark:block">
    <iframe
        src="https://diffusers-compute-pipeline-size.hf.space?__theme=dark"
        width="850"
        height="1600"
    ></iframe>
</div>

### 구성 요소별 데이터 유형 지정

`torch_dtype` 매개변수에 사전을 전달하여 개별 하위 모델의 데이터 유형을 사용자 지정할 수 있습니다. 이를 통해 파이프라인의 다른 구성 요소를 다른 부동 소수점 정밀도로 로드할 수 있습니다. 예를 들어 트랜с포머를 `torch.bfloat16`으로 로드하고 다른 모든 구성 요소를 `torch.float16`으로 로드하려면 매핑 사전을 전달할 수 있습니다.

In [ ]:
from diffusers import HunyuanVideoPipeline
import torch

pipe = HunyuanVideoPipeline.from_pretrained(
    "hunyuanvideo-community/HunyuanVideo",
    torch_dtype={"transformer": torch.bfloat16, "default": torch.float16},
)
print(pipe.transformer.dtype, pipe.vae.dtype)  # (torch.bfloat16, torch.float16)

사전에 구성 요소가 명시적으로 지정되지 않고 `default`가 제공되지 않으면 `torch.float32`로 로드됩니다.

### 로컬 파이프라인

로컬에서 파이프라인을 로드하려면 [git-lfs](https://git-lfs.github.com/)를 사용하여 체크포인트를 로컬 디스크에 수동으로 다운로드합니다.

```bash
git-lfs install
git clone https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5
```

이렇게 하면 디스크에 ./stable-diffusion-v1-5라는 로컬 폴더가 생성되며 해당 경로를 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained)에 전달해야 합니다.

In [ ]:
from diffusers import DiffusionPipeline

stable_diffusion = DiffusionPipeline.from_pretrained("./stable-diffusion-v1-5", use_safetensors=True)

[from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드는 로컬 경로를 감지하면 Hub에서 파일을 다운로드하지 않지만, 이는 체크포인트의 최신 변경 사항을 다운로드하고 캐시하지 않는다는 의미이기도 합니다.

## 파이프라인 사용자 지정

파이프라인에 다른 구성 요소를 로드하여 파이프라인을 사용자 지정할 수 있습니다. 이는 다음과 같은 이유로 중요합니다.

- 필요에 따라 생성 속도가 빠르거나 생성 품질이 높은 스케줄러로 변경할 수 있습니다(파이프라인에서 `scheduler.compatibles` 메서드를 호출하여 호환되는 스케줄러 확인).
- 기본 파이프라인 구성 요소를 더 새롭고 성능이 좋은 구성 요소로 변경할 수 있습니다.

예를 들어, 기본 [stabilityai/stable-diffusion-xl-base-1.0](https://hf.co/stabilityai/stable-diffusion-xl-base-1.0) 체크포인트를 다음과 같이 사용자 지정해 보겠습니다.

- [HeunDiscreteScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/heun#diffusers.HeunDiscreteScheduler)를 사용하여 생성 속도가 느려지는 대신 고품질 이미지를 생성합니다. 스케줄러 구성을 파이프라인 리포지토리의 올바른 [하위 폴더](https://hf.co/stabilityai/stable-diffusion-xl-base-1.0/tree/main/scheduler)에 로드하려면 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/schedulers/overview#diffusers.SchedulerMixin.from_pretrained)에서 `subfolder="scheduler"` 매개변수를 전달해야 합니다.
- fp16에서 실행되는 더 안정적인 VAE입니다.

In [ ]:
from diffusers import StableDiffusionXLPipeline, HeunDiscreteScheduler, AutoencoderKL
import torch

scheduler = HeunDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")
vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16, use_safetensors=True)

이제 새 스케줄러와 VAE를 [StableDiffusionXLPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/stable_diffusion_xl#diffusers.StableDiffusionXLPipeline)에 전달합니다.

In [ ]:
pipeline = StableDiffusionXLPipeline.from_pretrained(
  "stabilityai/stable-diffusion-xl-base-1.0",
  scheduler=scheduler,
  vae=vae,
  torch_dtype=torch.float16,
  variant="fp16",
  use_safetensors=True
).to("cuda")

## 파이프라인 재사용

동일한 모델 구성 요소를 공유하는 여러 파이프라인을 로드할 때, 특히 하드웨어 메모리가 제한적인 경우 모든 것을 메모리에 다시 로드하는 대신 공유 구성 요소를 재사용하는 것이 합리적입니다. 예를 들어,

1. [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)으로 이미지를 생성했지만 [StableDiffusionSAGPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/self_attention_guidance#diffusers.StableDiffusionSAGPipeline)으로 품질을 향상시키고 싶습니다. 이 두 파이프라인은 모두 동일한 사전 학습된 모델을 공유하므로 동일한 모델을 두 번 로드하는 것은 메모리 낭비입니다.
2. [`MotionAdapter`](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/animatediff#animatediffpipeline)와 같은 모델 구성 요소를 기존 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)에서 인스턴스화된 [AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)에 추가하고 싶습니다. 다시 말하지만, 두 파이프라인 모두 동일한 사전 학습된 모델을 공유하므로 완전히 새로운 파이프라인을 다시 로드하는 것은 메모리 낭비입니다.

[DiffusionPipeline.from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) API를 사용하면 메모리 사용량을 늘리지 않고 여러 파이프라인 간에 전환하여 다양한 기능을 활용할 수 있습니다. 파이프라인에서 기능을 켜고 끄는 것과 유사합니다.

> [!TIP]
> 기능이 아닌 작업 간에 전환하려면 [AutoPipeline](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/auto_pipeline) 클래스와 함께 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드를 사용하십시오. 이 클래스는 작업을 기반으로 파이프라인 클래스를 자동으로 식별합니다([AutoPipeline](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../tutorials/autopipeline) 튜토리얼에서 자세히 알아보십시오).

[StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)으로 시작한 다음 로드된 모델 구성 요소를 재사용하여 [StableDiffusionSAGPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/self_attention_guidance#diffusers.StableDiffusionSAGPipeline)을 만들어 생성 품질을 높여 보겠습니다. [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)과 [IP-Adapter](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./ip_adapter)를 사용하여 피자를 먹는 곰을 생성합니다.

In [ ]:
from diffusers import DiffusionPipeline, StableDiffusionSAGPipeline
import torch
import gc
from diffusers.utils import load_image
from accelerate.utils import compute_module_sizes

image = load_image("https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/load_neg_embed.png")

pipe_sd = DiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V6.0_B1_noVAE", torch_dtype=torch.float16)
pipe_sd.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")
pipe_sd.set_ip_adapter_scale(0.6)
pipe_sd.to("cuda")

generator = torch.Generator(device="cpu").manual_seed(33)
out_sd = pipe_sd(
    prompt="bear eats pizza",
    negative_prompt="wrong white balance, dark, sketches,worst quality,low quality",
    ip_adapter_image=image,
    num_inference_steps=50,
    generator=generator,
).images[0]
out_sd

<div class="flex justify-center">
  <img class="rounded-xl" src="https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/from_pipe_out_sd_0.png"/>
</div>

참고로 이 프로세스에서 소비된 메모리 양을 확인할 수 있습니다.

In [ ]:
def bytes_to_giga_bytes(bytes):
    return bytes / 1024 / 1024 / 1024
print(f"Max memory allocated: {bytes_to_giga_bytes(torch.cuda.max_memory_allocated())} GB")
"Max memory allocated: 4.406213283538818 GB"

이제 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe) 메서드를 사용하여 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)의 동일한 파이프라인 구성 요소를 [StableDiffusionSAGPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/self_attention_guidance#diffusers.StableDiffusionSAGPipeline)에서 재사용합니다.

> [!WARNING]
> 일부 파이프라인 메서드는 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe)로 생성된 새 파이프라인에서 제대로 작동하지 않을 수 있습니다. 예를 들어 [enable_model_cpu_offload()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.enable_model_cpu_offload) 메서드는 각 파이프라인에 대한 고유한 오프로딩 시퀀스를 기반으로 모델 구성 요소에 후크를 설치합니다. 모델이 새 파이프라인에서 다른 순서로 실행되면 CPU 오프로딩이 제대로 작동하지 않을 수 있습니다.
>
> 모든 것이 예상대로 작동하도록 하려면 [from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe)로 생성된 새 파이프라인에 파이프라인 메서드를 다시 적용하는 것이 좋습니다.

In [ ]:
pipe_sag = StableDiffusionSAGPipeline.from_pipe(
    pipe_sd
)

generator = torch.Generator(device="cpu").manual_seed(33)
out_sag = pipe_sag(
    prompt="bear eats pizza",
    negative_prompt="wrong white balance, dark, sketches,worst quality,low quality",
    ip_adapter_image=image,
    num_inference_steps=50,
    generator=generator,
    guidance_scale=1.0,
    sag_scale=0.75
).images[0]
out_sag

<div class="flex justify-center">
  <img class="rounded-xl" src="https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/from_pipe_out_sag_1.png"/>
</div>

메모리 사용량을 확인하면 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)과 [StableDiffusionSAGPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/self_attention_guidance#diffusers.StableDiffusionSAGPipeline)이 동일한 파이프라인 구성 요소를 공유하기 때문에 이전과 동일하게 유지되는 것을 알 수 있습니다. 이를 통해 추가적인 메모리 오버헤드 없이 상호 교환적으로 사용할 수 있습니다.

In [ ]:
print(f"Max memory allocated: {bytes_to_giga_bytes(torch.cuda.max_memory_allocated())} GB")
"Max memory allocated: 4.406213283538818 GB"

[AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)으로 이미지를 애니메이션화하고 파이프라인에 `MotionAdapter` 모듈도 추가해 보겠습니다. [AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)의 경우 먼저 IP-Adapter를 언로드한 다음 새 파이프라인을 만든 *후에* 다시 로드해야 합니다(이는 [AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)에만 적용됨).

In [ ]:
from diffusers import AnimateDiffPipeline, MotionAdapter, DDIMScheduler
from diffusers.utils import export_to_gif

pipe_sag.unload_ip_adapter()
adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2", torch_dtype=torch.float16)

pipe_animate = AnimateDiffPipeline.from_pipe(pipe_sd, motion_adapter=adapter)
pipe_animate.scheduler = DDIMScheduler.from_config(pipe_animate.scheduler.config, beta_schedule="linear")
# IP-Adapter 및 LoRA 가중치를 다시 로드합니다.
pipe_animate.load_ip_adapter("h94/IP-Adapter", subfolder="models", weight_name="ip-adapter_sd15.bin")
pipe_animate.load_lora_weights("guoyww/animatediff-motion-lora-zoom-out", adapter_name="zoom-out")
pipe_animate.to("cuda")

generator = torch.Generator(device="cpu").manual_seed(33)
pipe_animate.set_adapters("zoom-out", adapter_weights=0.75)
out = pipe_animate(
    prompt="bear eats pizza",
    num_frames=16,
    num_inference_steps=50,
    ip_adapter_image=image,
    generator=generator,
).frames[0]
export_to_gif(out, "out_animate.gif")

<div class="flex justify-center">
  <img class="rounded-xl" src="https://huggingface.co/datasets/YiYiXu/testing-images/resolve/main/from_pipe_out_animate_3.gif"/>
</div>

[AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)은 메모리 집약적이며 15GB의 메모리를 소비합니다(이것이 메모리 사용량에 어떤 의미인지 알아보려면 [from_pipe의 메모리 사용량](#memory-usage-of-from_pipe) 섹션을 참조하십시오).

In [ ]:
print(f"Max memory allocated: {bytes_to_giga_bytes(torch.cuda.max_memory_allocated())} GB")
"Max memory allocated: 15.178664207458496 GB"

### from_pipe 구성 요소 수정

[from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe)로 로드된 파이프라인은 다른 모델 구성 요소 또는 메서드로 사용자 지정할 수 있습니다. 그러나 모델 구성 요소의 *상태*를 수정할 때마다 동일한 구성 요소를 공유하는 다른 모든 파이프라인에 영향을 미칩니다. 예를 들어, [StableDiffusionSAGPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/self_attention_guidance#diffusers.StableDiffusionSAGPipeline)에서 [unload_ip_adapter()](https://huggingface.co/docs/diffusers/main/en/api/loaders/ip_adapter#diffusers.loaders.IPAdapterMixin.unload_ip_adapter)를 호출하면 공유 구성 요소에서 제거되었기 때문에 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)과 함께 IP-Adapter를 사용할 수 없습니다.

In [ ]:
pipe.sag_unload_ip_adapter()

generator = torch.Generator(device="cpu").manual_seed(33)
out_sd = pipe_sd(
    prompt="bear eats pizza",
    negative_prompt="wrong white balance, dark, sketches,worst quality,low quality",
    ip_adapter_image=image,
    num_inference_steps=50,
    generator=generator,
).images[0]
"AttributeError: 'NoneType' object has no attribute 'image_projection_layers'"

### from_pipe의 메모리 사용량

[from_pipe()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pipe)를 사용하여 여러 파이프라인을 로드하는 데 필요한 메모리는 생성하는 파이프라인 수에 관계없이 메모리 사용량이 가장 높은 파이프라인에 의해 결정됩니다.

| 파이프라인 | 메모리 사용량 (GB) |
|---|---|
| StableDiffusionPipeline | 4.400 |
| StableDiffusionSAGPipeline | 4.400 |
| AnimateDiffPipeline | 15.178 |

[AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)은 메모리 요구 사항이 가장 높으므로 *총 메모리 사용량*은 [AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)에만 기반합니다. 메모리 요구 사항이 [AnimateDiffPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/animatediff#diffusers.AnimateDiffPipeline)의 요구 사항을 초과하지 않는 한 추가 파이프라인을 만들어도 메모리 사용량이 증가하지 않습니다. 각 파이프라인은 추가적인 메모리 오버헤드 없이 상호 교환적으로 사용할 수 있습니다.

## 안전 검사기

Diffusers는 유해한 콘텐츠를 생성할 수 있는 Stable Diffusion 모델에 대한 [안전 검사기](https://github.com/huggingface/diffusers/blob/main/src/diffusers/pipelines/stable_diffusion/safety_checker.py)를 구현합니다. 안전 검사기는 생성된 출력을 알려진 하드코딩된 NSFW(Not-Safe-For-Work) 콘텐츠와 비교하여 검사합니다. 어떤 이유로든 안전 검사기를 비활성화하려면 [from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained) 메서드에 `safety_checker=None`을 전달합니다.

In [ ]:
from diffusers import DiffusionPipeline

pipeline = DiffusionPipeline.from_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", safety_checker=None, use_safetensors=True)
"""
`safety_checker=None`을 전달하여 <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion.StableDiffusionPipeline'>에 대한 안전 검사기를 비활성화했습니다. Stable Diffusion 라이선스의 조건을 준수하고 필터링되지 않은 결과를 공개 서비스 또는 애플리케이션에 노출하지 않도록 하십시오. diffusers 팀과 Hugging Face는 모든 공개 상황에서 안전 필터를 활성화하고 네트워크 동작 분석 또는 결과 감사와 관련된 사용 사례에 대해서만 비활성화하는 것을 강력히 권장합니다. 자세한 내용은 https://github.com/huggingface/diffusers/pull/254를 참조하십시오.
"""

## 체크포인트 변형

체크포인트 변형은 일반적으로 다음과 같은 가중치를 가진 체크포인트입니다.

- 대역폭과 저장 공간을 절반만 필요로 하기 때문에 [torch.float16](https://pytorch.org/docs/stable/tensors.html#data-types)과 같은 다른 부동 소수점 유형으로 저장됩니다. 학습을 계속하거나 CPU를 사용하는 경우 이 변형을 사용할 수 없습니다.
- 추론에 사용해서는 안 되는 비지수 평균(EMA) 가중치입니다. 모델을 계속 미세 조정하려면 이 변형을 사용해야 합니다.

> [!TIP]
> 체크포인트에 동일한 모델 구조가 있지만 다른 데이터 세트와 다른 학습 설정으로 학습된 경우 별도의 리포지토리에 저장해야 합니다. 예를 들어 [stabilityai/stable-diffusion-2](https://hf.co/stabilityai/stable-diffusion-2)와 [stabilityai/stable-diffusion-2-1](https://hf.co/stabilityai/stable-diffusion-2-1)은 별도의 리포지토리에 저장됩니다.

그렇지 않으면 변형은 원본 체크포인트와 **동일**합니다. 직렬화 형식([safetensors](https://huggingface.co/docs/diffusers/main/en/using-diffusers/./using_safetensors)와 같음), 모델 구조가 정확히 동일하며 가중치의 텐서 모양도 동일합니다.

| **체크포인트 유형** | **가중치 이름**                             | **가중치 로드 인수** |
|---------------------|---------------------------------------------|----------------------------------|
| 원본            | diffusion_pytorch_model.safetensors         |                                  |
| 부동 소수점      | diffusion_pytorch_model.fp16.safetensors    | `variant`, `torch_dtype`         |
| 비 EMA             | diffusion_pytorch_model.non_ema.safetensors | `variant`                        |

변형을 로드하는 데 중요한 두 가지 인수가 있습니다.

- `torch_dtype`은 로드된 체크포인트의 부동 소수점 정밀도를 지정합니다. 예를 들어 fp16 변형을 로드하여 대역폭을 절약하려면 `variant="fp16"` 및 `torch_dtype=torch.float16`을 설정하여 *가중치를 fp16으로 변환*해야 합니다. 그렇지 않으면 fp16 가중치가 기본 fp32 정밀도로 변환됩니다.

  `torch_dtype=torch.float16`만 설정하면 기본 fp32 가중치가 먼저 다운로드된 다음 fp16으로 변환됩니다.

- `variant`는 리포지토리에서 로드해야 하는 파일을 지정합니다. 예를 들어 [stable-diffusion-v1-5/stable-diffusion-v1-5](https://hf.co/stable-diffusion-v1-5/stable-diffusion-v1-5/tree/main/unet)에서 UNet의 비 EMA 변형을 로드하려면 `variant="non_ema"`를 설정하여 `non_ema` 파일을 다운로드합니다.

<hfoptions id="variants">
<hfoption id="fp16">

In [ ]:
from diffusers import DiffusionPipeline
import torch

pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", variant="fp16", torch_dtype=torch.float16, use_safetensors=True
)

</hfoption>
<hfoption id="non-EMA">

In [ ]:
pipeline = DiffusionPipeline.from_pretrained(
    "stable-diffusion-v1-5/stable-diffusion-v1-5", variant="non_ema", use_safetensors=True
)

</hfoption>
</hfoptions>

[DiffusionPipeline.save_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.save_pretrained) 메서드의 `variant` 매개변수를 사용하여 체크포인트를 다른 부동 소수점 유형 또는 비 EMA 변형으로 저장합니다. 원본 체크포인트와 동일한 폴더에 변형을 저장하여 동일한 폴더에서 둘 다 로드할 수 있도록 해야 합니다.

<hfoptions id="save">
<hfoption id="fp16">

In [ ]:
from diffusers import DiffusionPipeline

pipeline.save_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", variant="fp16")

</hfoption>
<hfoption id="non_ema">

In [ ]:
pipeline.save_pretrained("stable-diffusion-v1-5/stable-diffusion-v1-5", variant="non_ema")

</hfoption>
</hfoptions>

기존 폴더에 변형을 저장하지 않으면 원본 체크포인트를 찾을 수 없다는 `Exception`이 발생하므로 `variant` 인수를 지정해야 합니다.

In [ ]:
# 👎 작동하지 않습니다.
pipeline = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-v1-5", torch_dtype=torch.float16, use_safetensors=True
)
# 👍 작동합니다.
pipeline = DiffusionPipeline.from_pretrained(
    "./stable-diffusion-v1-5", variant="fp16", torch_dtype=torch.float16, use_safetensors=True
)

## DiffusionPipeline 설명

클래스 메서드로서 [DiffusionPipeline.from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained)는 다음 두 가지 작업을 수행합니다.

- 추론에 필요한 폴더 구조의 최신 버전을 다운로드하고 캐시합니다. 로컬 캐시에 최신 폴더 구조가 있는 경우 [DiffusionPipeline.from_pretrained()](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline.from_pretrained)는 캐시를 재사용하고 파일을 다시 다운로드하지 않습니다.
- 캐시된 가중치를 `model_index.json` 파일에서 검색된 올바른 파이프라인 [클래스](https://huggingface.co/docs/diffusers/main/en/using-diffusers/../api/pipelines/overview#diffusers-summary)로 로드하고 해당 인스턴스를 반환합니다.

파이프라인의 기본 폴더 구조는 해당 클래스 인스턴스와 직접적으로 일치합니다. 예를 들어 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)은 [`stable-diffusion-v1-5/stable-diffusion-v1-5`](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5)의 폴더 구조에 해당합니다.

In [ ]:
from diffusers import DiffusionPipeline

repo_id = "stable-diffusion-v1-5/stable-diffusion-v1-5"
pipeline = DiffusionPipeline.from_pretrained(repo_id, use_safetensors=True)
print(pipeline)

파이프라인이 [StableDiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)의 인스턴스임을 알 수 있으며, 이 인스턴스는 다음 7가지 구성 요소로 구성됩니다.

- `"feature_extractor"`: 🤗 Transformers의 [CLIPImageProcessor](https://huggingface.co/docs/transformers/main/en/model_doc/clip#transformers.CLIPImageProcessor)입니다.
- `"safety_checker"`: 유해한 콘텐츠를 검사하는 [구성 요소](https://github.com/huggingface/diffusers/blob/e55687e1e15407f60f32242027b7bb8170e58266/src/diffusers/pipelines/stable_diffusion/safety_checker.py#L32)입니다.
- `"scheduler"`: [PNDMScheduler](https://huggingface.co/docs/diffusers/main/en/api/schedulers/pndm#diffusers.PNDMScheduler)의 인스턴스입니다.
- `"text_encoder"`: 🤗 Transformers의 [CLIPTextModel](https://huggingface.co/docs/transformers/main/en/model_doc/clip#transformers.CLIPTextModel)입니다.
- `"tokenizer"`: 🤗 Transformers의 [CLIPTokenizer](https://huggingface.co/docs/transformers/main/en/model_doc/clip#transformers.CLIPTokenizer)입니다.
- `"unet"`: [UNet2DConditionModel](https://huggingface.co/docs/diffusers/main/en/api/models/unet2d-cond#diffusers.UNet2DConditionModel)의 인스턴스입니다.
- `"vae"`: [AutoencoderKL](https://huggingface.co/docs/diffusers/main/en/api/models/autoencoderkl#diffusers.AutoencoderKL)의 인스턴스입니다.

```json
StableDiffusionPipeline {
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}
```

파이프라인 인스턴스의 구성 요소를 [`stable-diffusion-v1-5/stable-diffusion-v1-5`](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/tree/main) 폴더 구조와 비교하면 리포지토리에 각 구성 요소에 대한 별도의 폴더가 있음을 알 수 있습니다.

```
.
├── feature_extractor
│   └── preprocessor_config.json
├── model_index.json
├── safety_checker
│   ├── config.json
|   ├── model.fp16.safetensors
│   ├── model.safetensors
│   ├── pytorch_model.bin
|   └── pytorch_model.fp16.bin
├── scheduler
│   └── scheduler_config.json
├── text_encoder
│   ├── config.json
|   ├── model.fp16.safetensors
│   ├── model.safetensors
│   |── pytorch_model.bin
|   └── pytorch_model.fp16.bin
├── tokenizer
│   ├── merges.txt
│   ├── special_tokens_map.json
│   ├── tokenizer_config.json
│   └── vocab.json
├── unet
│   ├── config.json
│   ├── diffusion_pytorch_model.bin
|   |── diffusion_pytorch_model.fp16.bin
│   |── diffusion_pytorch_model.f16.safetensors
│   |── diffusion_pytorch_model.non_ema.bin
│   |── diffusion_pytorch_model.non_ema.safetensors
│   └── diffusion_pytorch_model.safetensors
|── vae
.   ├── config.json
.   ├── diffusion_pytorch_model.bin
    ├── diffusion_pytorch_model.fp16.bin
    ├── diffusion_pytorch_model.fp16.safetensors
    └── diffusion_pytorch_model.safetensors
```

파이프라인의 각 구성 요소에 속성으로 액세스하여 구성을 볼 수 있습니다.

In [ ]:
pipeline.tokenizer
CLIPTokenizer(
    name_or_path="/root/.cache/huggingface/hub/models--runwayml--stable-diffusion-v1-5/snapshots/39593d5650112b4cc580433f6b0435385882d819/tokenizer",
    vocab_size=49408,
    model_max_length=77,
    is_fast=False,
    padding_side="right",
    truncation_side="right",
    special_tokens={
        "bos_token": AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
        "eos_token": AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
        "unk_token": AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True),
        "pad_token": "<|endoftext|>",
    },
    clean_up_tokenization_spaces=True
)

모든 파이프라인은 [DiffusionPipeline](https://huggingface.co/docs/diffusers/main/en/api/pipelines/overview#diffusers.DiffusionPipeline)에 다음을 알려주는 [`model_index.json`](https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/blob/main/model_index.json) 파일을 예상합니다.

- `_class_name`에서 로드할 파이프라인 클래스
- `_diffusers_version`에서 모델을 만드는 데 사용된 🧨 Diffusers 버전
- 하위 폴더에 저장된 라이브러리의 구성 요소(`name`은 구성 요소 및 하위 폴더 이름에 해당하고 `library`는 클래스를 로드할 라이브러리 이름에 해당하며 `class`는 클래스 이름에 해당함)

```json
{
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.6.0",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}
```